In [22]:
import pandas as pd
import requests
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Extraindo dados da API da NOAA
Como o dataset está disponível apenas através da API disponibilizada pela NOAA *(National Oceanic and Atmospheric Administration)*, foi preciso registrar um **token** no site da agência, que foi enviado por email, para conseguir acessar a API

In [23]:
#config
token = "WZzNRktcksOxuQRDpDgdTyqlNgVkqtBd"
endpoint = "https://www.ncei.noaa.gov/access/services/data/v1"

params = {
    "dataset": "daily-summaries",
    "stations": "USW00023174",
    "startDate": "2024-03-07",
    "endDate": "2025-10-31",
    "format": "csv",
    "units": "metric"
}

#request
response = requests.get(endpoint, params=params, headers={"token": token})


Salvar o arquivo extraído da API em um .csv na pasta */data*

In [24]:
from pathlib import Path

root = Path.cwd() #sobe um nível (.parent) pra chegar na raiz v 
data_dir = root/ "data" #aponta pra pasta data/

save_csv = data_dir / "LA-wheather-daily-2024-2025.csv" #salva na pasta data/ 

#salva o arquivo extraído da API na pasta
with open (save_csv, "wb") as f:
    f.write(response.content)
    
print("Arquivo salvo em: ", save_csv)

df_global_hourly = pd.read_csv(r"data\LA-wheather-daily-2024-2025.csv")

Arquivo salvo em:  d:\gabri\Documents\IFSP\5_semestre\SPOESTA\projeto\data\LA-wheather-daily-2024-2025.csv


## Análise estatística

### Perguntas que serão respondidas com as análises:  
  
1. A média de crime diário aumenta quando está calor? (mínimo de 25°C)  
2. Qual a probabilidade de acontecer um crime quando chove?  
3. Qual a probabilidade de um crime ocorrer em dias quentes?  
4. A dispersão de crimes muda entre dias quentes e dias frios?  
5. Qual tipo de crime é mais provável de acontecer em dias frios?

In [ ]:
#variáveis para os datasets

nibrs_offenses = pd.read_csv(r"data\LAPD_NIBRS_Offenses_Dataset.csv")
#nibrs_victims = pd.read_csv(r"data\LAPD_NIBRS_Victims_Dataset.csv")
wheather = pd.read_csv(r"data\LA-wheather-daily-2024-2025.csv")

## Limpeza e organização dos dados - NIBRS Offenses

In [ ]:
print(nibrs_offenses.dtypes)

In [ ]:
# alterando o tipo de dado para conseguir iniciar as análises

nibrs_offenses['Date OCC'] = pd.to_datetime(nibrs_offenses['Date OCC'])
nibrs_offenses['Date'] = nibrs_offenses['Date OCC'].dt.date #extrai apenas a data p/ comparar com o outro dataset que só tem yyyy-mm-dd

#criando a coluna de contagem de crimes por dia
daily_crimes = (
    nibrs_offenses.groupby('Date')['TotalOffenseCount'].sum() #agrupa a coluna ['TotalOffenseCount'] pelo critério ('Date') e soma tudo
    .reset_index() #pq 'Date' virou o índice
)
daily_crimes.columns = ['date', 'crime_count'] 

#pd.set_option("display.max_rows", None)
#daily_crimes

#daily_crimes.isna().sum()

## Limpeza e organização dos dados - NIBRS Offenses

In [ ]:
print(wheather.dtypes)

In [ ]:
wheather['DATE'] = pd.to_datetime(wheather['DATE'])
wheather['Date'] = wheather['DATE'].dt.date

dia_quente = 25 #25 graus Celsius

#criando lógica de dias quentes, chuvosos e frios
wheather['hot_day'] = wheather['TMAX'] >= dia_quente
wheather['cold_day'] = wheather['TMAX'] < dia_quente
wheather['rainy_day'] = wheather['PRCP'] > 0

#wheather['hot_day'].value_counts
#wheather['cold_day'].value_counts
#wheather['rainy_day'].value_counts

## Merge dos datasets

In [82]:
#INNER JOIN --> traz tudo das duas tabelas onde a data seja a mesma
final_df = daily_crimes.merge(
    wheather[['Date', 'TMAX', 'PRCP', 'hot_day', 'cold_day', 'rainy_day']],
    left_on='date',
    right_on='Date',
    how='inner'
)

final_df = final_df.drop(columns=['Date'])

#final_df.shape()
final_df.head()

,date,crime_count,TMAX,PRCP,hot_day,cold_day,rainy_day
0,2024-03-07,102,16.7,0.8,False,True,True
1,2024-03-08,83,20.6,0.0,False,True,False
2,2024-03-09,93,18.9,0.0,False,True,False
3,2024-03-10,118,17.8,0.0,False,True,False
4,2024-03-11,102,16.7,0.0,False,True,False
